# Verification for Asymmetric Architect Strategy

This notebook verifies that the population Jacobian remains diagonal at a vertex equilibrium even when the resident Architect strategy is asymmetric (P = p, R = r with p ≠ r), as stated in Lemma 4.3.

In [ ]:
from sympy import symbols, Matrix, simplify, diff, init_printing
init_printing()

# Parameters
a_tilde, b, gamma, c, p, r = symbols('a_tilde b gamma c p r')
# Mutant frequencies
yT, yV = symbols('yT yV')
# Resident frequency = 1 - yT - yV

# Payoffs for asymmetric resident (P=p, R=r)
# Using the general payoff function from Eq. (1)
# For interactions with resident (p,r):
Pi_AA = a_tilde*p*(1-p) - b*p*r + gamma*p*r - c*p*(1-r)
Pi_TA = a_tilde*(1-p) + gamma*r - c
Pi_VA = -b*r

# Payoffs among mutants (needed for fitness calculation)
Pi_TT = -c               # (1,0) vs (1,0)
Pi_TV = a_tilde + gamma - c  # (1,0) vs (0,1)
Pi_VT = 0                # (0,1) vs (1,0)
Pi_VV = -b               # (0,1) vs (0,1)

# Payoffs of mutants against each other (not needed at vertex but included for completeness)
# Payoffs of resident against mutants:
Pi_AT = -c*p*(1-r)
Pi_AV = a_tilde*p - b*r + gamma*r - c*p*(1-r)

# Fitness functions
f_T = yT*Pi_TT + yV*Pi_TV + (1 - yT - yV)*Pi_TA
f_V = yT*Pi_VT + yV*Pi_VV + (1 - yT - yV)*Pi_VA
f_A = yT*Pi_AT + yV*Pi_AV + (1 - yT - yV)*Pi_AA

# Average fitness
f_bar = yT*f_T + yV*f_V + (1 - yT - yV)*f_A

# Replicator equations
dyT_dt = yT * (f_T - f_bar)
dyV_dt = yV * (f_V - f_bar)

print('Replicator equations for asymmetric resident:')
print('dyT/dt =')
simplify(dyT_dt)

In [ ]:
# Compute the population Jacobian
J = Matrix([[diff(dyT_dt, yT), diff(dyT_dt, yV)],
            [diff(dyV_dt, yT), diff(dyV_dt, yV)]])

print('General population Jacobian (symbolic):')
J_simplified = simplify(J)
J_simplified

In [ ]:
# Evaluate at the vertex equilibrium (yT=0, yV=0)
J_at_vertex = simplify(J.subs({yT: 0, yV: 0}))
print('Jacobian at vertex (yT=0, yV=0):')
J_at_vertex

In [ ]:
# Show that off-diagonal entries are zero
print('Off-diagonal entries:')
print('J[0,1] =', J_at_vertex[0,1])
print('J[1,0] =', J_at_vertex[1,0])

print('\nDiagonal entries:')
print('J[0,0] =', simplify(J_at_vertex[0,0]))
print('Which equals Pi_TA - Pi_AA =', simplify(Pi_TA - Pi_AA))

print('\nJ[1,1] =', simplify(J_at_vertex[1,1]))
print('Which equals Pi_VA - Pi_AA =', simplify(Pi_VA - Pi_AA))

## Verification of Lemma 4.3

The Jacobian is diagonal at the vertex. The eigenvalues are the payoff differences, regardless of whether p = r (symmetric) or p ≠ r (asymmetric).

In [ ]:
# Check the specific case where p = r (should recover symmetric case)
print('Special case: symmetric resident p = r')
J_symmetric = simplify(J_at_vertex.subs({r: p}))
print('Jacobian at vertex with p = r:')
J_symmetric

In [ ]:
# Compute eigenvalues in general case
eigenvals = J_at_vertex.eigenvals()
print('Eigenvalues of Jacobian at vertex:')
eigenvals

## Implications for stability bounds

For asymmetric resident, the stability conditions are still:
1. Π_AA > Π_TA
2. Π_AA > Π_VA

Solving these inequalities for c would give bounds L(p,r) and U(p,r) that are functions of both p and r.

In [ ]:
# Example: derive the bound L for asymmetric case from Π_AA > Π_TA
print('Solving Π_AA > Π_TA for c:')
from sympy import solve, Lt
ineq = Lt(Pi_TA, Pi_AA)  # Π_TA < Π_AA
solution = solve(ineq, c)
print('Solution for c:', solution)

# The lower bound L would be the right-hand side of this inequality
L_asym = solution.as_set().args[1].args[0]
print('Lower bound L(p,r) =', simplify(L_asym))

## Conclusion

The diagonal structure of the population Jacobian at a vertex equilibrium holds for any resident strategy (symmetric or asymmetric). This structural result is independent of the specific parameter values and relies only on the vertex property (mutant frequencies are zero at equilibrium).